#Main

read in and edit the compounds in data.xls

In [1]:
import periodictable as PT
import numpy as np
from collections import OrderedDict as OD
import pandas as pd
import numpy as np

In [2]:
def clean_string(s):
    #get rid of spaces
    s=s.replace(" ","")
    #get rid of funny plus sign
    s=s.replace('\xc2\xb7','+')
    #change [ to (
    s=s.replace('[','(')
    #change ] to )
    s=s.replace(']',')')
    return s
    

#check elements

In [3]:
#read elements from xls
df_elements=pd.read_excel('./data.xlsx','Elements',index_col=None,skiprows=1)
#write elements to csv
df_elements.to_csv('./element.csv',index=False,encoding='utf-8')

In [5]:
#read elements from csv
df_elements=pd.read_csv('./element.csv',encoding='utf-8')

In [6]:
#check elements
error_index=[]
eps=1e-6
for i,g in df_elements.iterrows():
    
    MW_1=g['Atomic Mass']
    
    
    try:
        MW_2=PT.elements.symbol(g.Symbol).mass
        diff=np.abs(MW_1-MW_2)/MW_1
        
        if diff>eps:
            PT.elements.symbol(g['Symbol'])._mass=MW_1
        
        
    except:
        MW_2=None
        diff=100
        
        
    
    if diff>eps:
        d=dict(symbol=g['Symbol'],number=g['Atomic Number'], \
               MW_1=MW_1,MW_2=MW_2,diff=diff)
        error_index.append(d)

    
pd.DataFrame(error_index)

,MW_1,MW_2,diff,number,symbol
0,97.9072,98.000,0.000948,43,Tc
1,144.2420,144.240,0.000014,60,Nd
2,144.9127,145.000,0.000602,61,Pm
3,195.0840,195.078,0.000031,78,Pt
4,208.9824,209.000,0.000084,84,Po
5,209.9871,210.000,0.000061,85,At
6,222.0176,222.000,0.000079,86,Rn
7,223.0197,223.000,0.000088,87,Fr
8,226.0254,226.000,0.000112,88,Ra
9,227.0277,227.000,0.000122,89,Ac


In [7]:
#adjust elements
for i,g in df_elements.iterrows():
    
    MW_1=g['Atomic Mass']
    try:
        e=PT.elements.symbol(g.Symbol)
        e._mass=MW_1
    except:
        print "element %s not present"%(g.Symbol)
    

element Uub not present


#functions

In [8]:
def load_sheet(file_name,sheet_name,index_offset=2):
    df=pd.read_excel(file_name,sheet_name)
    
    #alter index
    df.index=df.index+index_offset
    
    #rename formula
    #df=df.rename(columns={'Formula':'str_Formula'})
    
    return df

def check_molecular_weigths(df,eps=1e-5):
    error_index=[]
    
    for i,g in df.iterrows():
        MW_1=g['MW']
        try:
            f=PT.formula(g['Formula'])
            MW_2=f.mass
            diff=np.abs(MW_1-MW_2)/MW_1
        except:
            f=None
            MW_2=-1.0
            diff=100.0

        #get MW
        diff=np.abs(MW_1-MW_2)/MW_1

        if diff>eps:
            d=dict(index=i,Formula=g['Formula'],MW_1=MW_1,MW_2=MW_2,diff=diff)
            error_index.append(d)

    
    return pd.DataFrame(error_index)    

def f_str_Elements(df):
    return df.apply(lambda x: unicode([str(y) for y in PT.formula(x['Formula']).atoms.keys()]),axis=1)    

def clean_string(s):
    #get rid of spaces
    s=s.replace(" ","")
    #change [ to (
    s=s.replace('[','(')
    #change ] to )
    s=s.replace(']',')')
    return s


def add_elements(df,c='Formula',cnew='source'):
    """
    add in and stack element names
    """
    
    #get all the elements as columns
    x = df[c].apply(lambda x:pd.Series([str(y) for y in PT.formula(x).atoms.keys()]))
    #stack and drop bottom level
    x = x.stack().reset_index(level=-1,drop=True)
    #make a Dataframe with new column name
    x = pd.DataFrame({cnew:x})
    #merge in 
    t = df.copy()
    return pd.concat((t,x),axis=1)
    
    

In [9]:
def add_elements(df,c='Formula',cnew='Target',mass_frac_name='mass_frac'):
    """
    add in and stack element names
    """
    
 
    #get all the elements as columns
    x = df[c].apply(lambda x:pd.Series({str(k):v for k,v in PT.formula(x).mass_fraction.iteritems()}))

    #stack, frame, drop bottom level to column, add name'
    x = pd.DataFrame({mass_frac_name:x.stack()}).reset_index(level=-1).rename(columns={'level_1':cnew})    

    #merge in 
    t = df.copy()
    return pd.concat((t,x),axis=1)
    

make a csv file with the following basic data

 * 'Name': name of compound (nan if not present)
 * 'Formula': string representation of formula
 * 'CAS': cas number (nan if not present)
 * 'Target': string rep for element if element in 'formula'
 * 'MassFrac': mass fraction of given 'Targe'
 

#Acids

In [10]:
#for acid calculations, do something special.
#consider acids in there pure form (no water) in calculation, then add in mass % of acid
#this assumes that all the water will evaporate off.

In [11]:
df = load_sheet('./data.xlsx','Acids')

In [12]:
df.to_csv('acid.csv',index=False,encoding='utf-8')

In [13]:
def get_molality(solute,solvent,mf_solute):
    """
    returns molality (mol/g) of solute in solvent with mass fraction of solute=mf_solute
    """
    if type(solute) is not PT.formulas.Formula:
        solute=PT.formula(solute)
    if type(solvent) is not PT.formulas.Formula:
        solvent=PT.formula(solvent)
        
    mol_solute=mf_solute/solute.mass
    mass_solvent=(1.-mf_solute)
    return mol_solute/mass_solvent

In [14]:
t = add_elements(df)

In [15]:
t.to_csv('acid.csv',index=False,encoding='utf-8')

#oxides

In [16]:
df=load_sheet('./data.xlsx','inorganic oxide')

In [17]:
check=check_molecular_weigths(df)
check

,Formula,MW_1,MW_2,diff,index
0,Ac2O3,502.000,502.053600,0.000107,2
1,AmO2,275.000,275.060200,0.000219,4
2,BeO,25.011,25.011582,0.000023,8
3,Li2O,29.881,29.881400,0.000013,35
4,NpO2,269.000,269.047000,0.000175,42
5,NiO,74.692,74.692800,0.000011,43
6,Pu2O3,536.000,536.126600,0.000236,50
7,PoO2,241.000,240.981200,0.000078,51
8,RuO4,165.070,165.067600,0.000015,54
9,SeO2,110.960,110.958800,0.000011,60


In [18]:
#all look good enough :)

In [19]:
#clean up Formulas
df['Formula']=df.apply(lambda x: clean_string(x['Formula']),axis=1)

In [20]:
def add_elements(df,c='Formula',cnew='Target',mass_frac_name='mass_frac'):
    """
    add in and stack element names
    """
    
 
    #get all the elements as columns
    x = df[c].apply(lambda x:pd.Series({str(k):v for k,v in PT.formula(x).mass_fraction.iteritems()}))

    #stack, frame, drop bottom level to column, add name'
    x = pd.DataFrame({mass_frac_name:x.stack()}).reset_index(level=-1).rename(columns={'level_1':cnew})    

    #merge in 
    t = df.copy()
    return pd.concat((t,x),axis=1)
    

In [21]:
df = add_elements(df)

In [22]:
df.to_csv('./oxide.csv',index=False,encoding='utf-8')

#inorganic salts

In [23]:
df=load_sheet('./data.xlsx','inorganic salts')

In [24]:
#clean up Formulas
df['Formula']=df.apply(lambda x: clean_string(x['Formula']),axis=1)

In [25]:
errors=check_molecular_weigths(df)

In [26]:
errors[errors['diff']>.0001]

,Formula,MW_1,MW_2,diff,index
0,Ac,227.0,227.027700,0.000122,2
2,Ac2O3,502.0,502.053600,0.000107,4
7,Am,243.0,243.061400,0.000253,23
8,AmF3,300.0,300.056610,0.000189,24
9,AmF4,319.0,319.055013,0.000172,25
10,AmI3,624.0,623.774810,0.000361,26
11,Am2O3,534.0,534.121000,0.000227,27
12,AmO2,275.0,275.060200,0.000219,28
21,Bk,247.0,247.070300,0.000285,115
24,Ca2PbO4,351.4,351.353600,0.000132,177


In [27]:
#this looks good to me

In [28]:
t = add_elements(df)

In [30]:
df = t
df.to_csv('./salt_1.csv',index=False,encoding='utf-8')

#inorganic salts (all)

In [31]:
df=load_sheet('./data.xlsx','inorganic salts (all)')

In [32]:
def clean_string(s):
    #get rid of spaces
    s=s.replace(" ","")
    #change [ to (
    s=s.replace('[','(')
    #change ] to )
    s=s.replace(']',')')
    
    if u'\xb7' in s:
        s=s.replace(u'\xb7','+')
    if u'\xc2' in s:
        s=s.replace(u'\xc2','')

    return s

In [33]:
#clean up Formulas
df['Formula']=df.apply(lambda x: clean_string(x['Formula']),axis=1)

In [34]:
#drop null MW
df=df.dropna(subset=['MW'])

In [35]:
errors=check_molecular_weigths(df)

error at index
373: No MW for cas

818:  All screwed up
1449: (no idea where MW_1 comes from.  MW on web is 565, corresponding to no waters: 'La2(SO4)3')
2549: (MW_1 correspnds to Na3Au(S2O3)2+(H2O)2, not formula Na3Au(S2O3)2+5H2O)

3152

error, but think I'm right (i.e., MW_1 is wrong):
1445, 1456, 1553, 2400, 2617, 2623, 370, 939




cleaned up:
804,883, 892,1184,1261,1430,

660: changed Ce(C2O4)3+9H2O to Ce2(C2O4)3+9H2O

1455: reworked formula from Pb(C2H3O2)2+2Pb(OH)2 -> Pb(C2H3O2)2+(Pb(OH)2)2

1857: changed formula Nd(C2H3O2)2 -> Nd(C2H3O2)3

3100: changes 3Zn(OH)2+2ZnCO3 -> (Zn(OH)2)3·2ZnCO3

no error (just rounded MW):
2111,2341,4,75,

In [36]:
bad_errors=list(errors[errors['diff']>.001]['index'].values)
print bad_errors

[4, 75, 370, 373, 818, 939, 1445, 1449, 1456, 1553, 2111, 2341, 2400, 2549, 2617, 2623, 3152]


In [37]:
rounding='2111,2341,4,75'.split(',')
fixed='804,883, 892,1184,1261,1430,660,1455,1857,3100'.split(',')
error_questionable='1445, 1456, 1553, 2400, 2617, 2623, 370, 939'.split(',')
screwed='373,818,1449,2549,3152'.split(',')

In [38]:
L=bad_errors[:]

for s in rounding:
    x=int(s)
    if x in L:
        print "removing",x
        L.remove(x)
    else:
        print "rounding",x,'not in L'
        
for s in fixed:
    x=int(s)
    if x in L:
        print "shit",x,"in L"
        
for s in error_questionable:
    x=int(s)
    if x in L:
        print "removing",x
        L.remove(x)
    else:
        print "question",x,'not in L'

for s in screwed:
    x=int(s)
    if x in L:
        print "removing",x
        L.remove(x)
    else:
        print "screwed",x,'not in L'
print L

removing 2111
removing 2341
removing 4
removing 75
removing 1445
removing 1456
removing 1553
removing 2400
removing 2617
removing 2623
removing 370
removing 939
removing 373
removing 818
removing 1449
removing 2549
removing 3152
[]


In [39]:
#remove screwed ones

In [40]:
drop_list=[int(x) for x in screwed]
df=df.drop(drop_list)

In [41]:
errors=check_molecular_weigths(df)

In [42]:
bad_errors=list(errors[errors['diff']>.001]['index'].values)

In [163]:
L=bad_errors[:]

for s in rounding:
    x=int(s)
    if x in L:
        print "removing",x
        L.remove(x)
    else:
        print "rounding",x,'not in L'
        
for s in fixed:
    x=int(s)
    if x in L:
        print "shit",x,"in L"
        
for s in error_questionable:
    x=int(s)
    if x in L:
        print "removing",x
        L.remove(x)
    else:
        print "question",x,'not in L'

for s in screwed:
    x=int(s)
    if x in L:
        print "removing",x
        L.remove(x)
    else:
        print "screwed",x,'not in L'
print L

removing 2111
removing 2341
removing 4
removing 75
removing 1445
removing 1456
removing 1553
removing 2400
removing 2617
removing 2623
removing 370
removing 939
screwed 373 not in L
screwed 818 not in L
screwed 1449 not in L
screwed 2549 not in L
screwed 3152 not in L
[]


In [54]:
t = add_elements(df)

In [55]:
df = t
df.to_csv('./salt.csv',index=False,encoding='utf-8')

#merge salts/salts_all

In [56]:
df1 = pd.read_csv('salt_1.csv',encoding='utf-8')
df2 = pd.read_csv('salt.csv',encoding='utf-8')

In [58]:
formula_1 = df1.Formula.unique()
formula_2 = df2.Formula.unique()

In [64]:
for f in formula_1:
    if f not in formula_2:
        print f

In [67]:
dfA = df1[['Formula','MW','Target','mass_frac']]
dfB = df2[['Formula','MW','Target','mass_frac']]

In [70]:
t = pd.merge(dfA,dfB,how='left',on=['Formula','Target'])